In [ ]:
# pip install parsel

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import urllib.request
import time
import io
import sys
import asyncio
import re
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import chromedriver_autoinstaller
path = chromedriver_autoinstaller.install()
from parsel import Selector
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


#exec_path = 'D:\\xref\\scrape\\webdriver\\chromedriver'
pd.set_option('display.max_colwidth', None)
driver = webdriver.Chrome(executable_path = path)

import warnings
warnings.filterwarnings("ignore")

# STARTUP_DELAY = 1
# NEXT_PAGE_DELAY =1
# WAIT_AFTER_ELEMENT_LOADED = 1
# DELAY_TO_CHECK_IF_MEW_ELEMENT_IS_VISIBLE =1
# LOAD_SPECIFIC_RESULT_DELAY = 1
# BACK_TO_MAIN_LIST_DELAY = 1

In [2]:
raw_text = ['Saint Louis County MO', 'Saint Charls County MO']
replace_dict = {"MO": "Missouri"}
for key, value in replace_dict.items():
    raw_text = [item.replace(key, value) for item in raw_text]
    
search_string_list = ["Physical Thrapy Clinic"]#, "Radiology Clinic"]

In [3]:
print("Raw text:\n", raw_text)

print("search string:\n", search_string_list)

Raw text:
 ['Saint Louis County Missouri', 'Saint Charls County Missouri']
search string:
 ['Physical Thrapy Clinic']


In [5]:
df_level_1 = pd.DataFrame(columns=["LocName","LocRate","LocBussType","LocBussAdd","LocHREF"])
junk_url_list = []


# keep_looping=True
# while  keep_looping:

N = 4

idx = 0

for prefix in raw_text:
    t1 = pd.Timestamp.now()
    df_level_1.to_pickle('Physical_Therapy_MO_level_1_scrape_intermediate.pkl')
    prefix = prefix.replace(' ',"+")
    print(prefix)
    for search_string in search_string_list:
        search_string = search_string.replace(' ',"+")
        print(search_string)

        url_local = search_string + '+' + prefix
        url_local = 'https://www.google.com/maps/search/' + url_local
        print(url_local)
        driver.get(url_local)

        time.sleep(1)

        
        
        keep_looping = True
        page_number = 0
        while page_number <= N:
            action = webdriver.ActionChains(driver)
            soup = BeautifulSoup(driver.page_source, 'lxml')
    #         print(soup)

            for button_tmp in soup.find_all('button'):
                try:

                    if button_tmp['aria-label'] == ' Next page ':
                        button_id = button_tmp['id']

                except:
                    pass

        
            python_button = driver.find_element_by_id(button_id)
            action.move_to_element(python_button)
            action.perform()
            action.move_by_offset(35, -50)
            action.double_click()
            action.perform()
            time.sleep(.25)
            action.double_click()
            action.perform()
            time.sleep(.25)
            action.double_click()
            action.perform()
            time.sleep(.25)
            action.double_click()
            action.perform()
            time.sleep(.25)


            page_content = driver.page_source
    #         print(page_content)


            response = Selector(page_content)
            results = []

            for el in response.xpath('//div[contains(@aria-label, "Results for")]/div/div[./a]'):
                LocHREF = el.xpath('./a/@href').extract_first('')
                df_level_1.loc[idx,"LocHREF"] = LocHREF

                LocName= el.xpath('./a/@aria-label').extract_first('')
                df_level_1.loc[idx,"LocName"] = LocName

                
                LocBussType= el.xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[19]/div/div[2]/div[2]/div[1]/div/div/div/div[4]/div[1]/span[1]/jsl/span[2]')
                LocBussType = LocBussType.extract_first('')
                df_level_1.loc[idx,"LocBussType"] = LocBussType
                
                
#                 df_level_1.loc[idx,"raw text"] = prefix
#                 df_level_1.loc[idx,"search string"] = search_string
                
                
    

                idx += 1

            if (page_number+1)%3==0:
                df_level_1.to_csv("google_map_result_{}_{}_{}.csv".format(prefix,search_string,int(page_number+1)))
                df_level_1 = pd.DataFrame(columns=["LocName","LocRate","LocBussType","LocBussAdd","LocHREF"])
                idx = 0
        
            page_number += 1
        
            if page_number <= N:
                try:
                    python_button = driver.find_element_by_id(button_id)
                    python_button.click()
                    time.sleep(0.75)
                except:
                    #print('reached end of good buttons')
                    # keep_looping = False
                    pass


        df_level_1.to_csv("google_map_result_{}_{}_{}.csv".format(prefix,search_string,int(page_number)))              
            
                
                
                
    #             nextpage = driver.find_element_by_xpath("/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div/button[2]/img")
    #             nextpage.click()
    #             time.sleep(5)


    #             results.append({
    #                 'link': el.xpath('./a/@href').extract_first(''),
    #                 'title': el.xpath('./a/@aria-label').extract_first(''),
    #                 'rate': el.xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[6]/div/div[2]/div[2]/div[1]/div/div/div/div[3]/div/span[2]/span[2]').extract_first(''),
    #                 'BusType': el.xpath('/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[26]/div/div[2]/div[2]/div[1]/div/div/div/div[4]/div[1]/span[1]/jsl/span[2]').extract_first('')

    #         })

    #         print(results)
df_level_1

Saint+Louis+County+Missouri
Physical+Thrapy+Clinic
https://www.google.com/maps/search/Physical+Thrapy+Clinic+Saint+Louis+County+Missouri
Saint+Charls+County+Missouri
Physical+Thrapy+Clinic
https://www.google.com/maps/search/Physical+Thrapy+Clinic+Saint+Charls+County+Missouri


,LocName,LocRate,LocBussType,LocBussAdd,LocHREF
0,Athletico Physical Therapy - Jennings,NaN,,NaN,https://www.google.com/maps/place/Athletico+Physical+Therapy+-+Jennings/data=!4m6!3m5!1s0x87df4a211498cc17:0x2bfad5a4d9598268!8m2!3d38.7294659!4d-90.2735579!16s%2Fg%2F1q6jz53n4?authuser=0&hl=en&rclk=1
1,"Scott D. Mathys, PT",NaN,,NaN,"https://www.google.com/maps/place/Scott+D.+Mathys,+PT/data=!4m6!3m5!1s0x87ded0e6b626a3d5:0xb14db86883139847!8m2!3d38.7815912!4d-90.700661!16s%2Fg%2F1vxzb4_3?authuser=0&hl=en&rclk=1"
2,"Leslie C. Tucker, OTR",NaN,,NaN,"https://www.google.com/maps/place/Leslie+C.+Tucker,+OTR/data=!4m6!3m5!1s0x87df28453057447d:0xc45a257af964cbf9!8m2!3d38.7893854!4d-90.5739996!16s%2Fg%2F1tl4dk_z?authuser=0&hl=en&rclk=1"
3,Owens Sarah A,NaN,,NaN,https://www.google.com/maps/place/Owens+Sarah+A/data=!4m6!3m5!1s0x87df2fab8342a9b3:0xc6518f6b258f36e6!8m2!3d38.780599!4d-90.483787!16s%2Fg%2F1tmgczrk?authuser=0&hl=en&rclk=1
4,SSM Health Physical Therapy,NaN,,NaN,https://www.google.com/maps/place/SSM+Health+Physical+Therapy/data=!4m6!3m5!1s0x87df32a44e6fdf37:0x67339bbdc7614147!8m2!3d38.6656206!4d-90.4438562!16s%2Fg%2F11f24x7ss_?authuser=0&hl=en&rclk=1
5,The Health Centers,NaN,,NaN,https://www.google.com/maps/place/The+Health+Centers/data=!4m6!3m5!1s0x87df2f2566904ead:0xc1ba726023f98323!8m2!3d38.7864115!4d-90.5159806!16s%2Fg%2F1vd3wbmf?authuser=0&hl=en&rclk=1
6,SSM Health Physical Therapy,NaN,,NaN,https://www.google.com/maps/place/SSM+Health+Physical+Therapy/data=!4m6!3m5!1s0x87ded7d9ea40b6c3:0x10686248404c1ce2!8m2!3d38.8041502!4d-90.6513523!16s%2Fg%2F11l_dt_22j?authuser=0&hl=en&rclk=1
7,Work Performance Center Inc,NaN,,NaN,https://www.google.com/maps/place/Work+Performance+Center+Inc/data=!4m6!3m5!1s0x87decf4426699c95:0x9cd3c3478002f509!8m2!3d38.8018307!4d-90.8170023!16s%2Fg%2F1ts2_ttc?authuser=0&hl=en&rclk=1
8,SSM Health Physical Therapy,NaN,,NaN,https://www.google.com/maps/place/SSM+Health+Physical+Therapy/data=!4m6!3m5!1s0x87dee70f406badd5:0x6b04bd6ffeb25ed8!8m2!3d38.9913301!4d-90.7475722!16s%2Fg%2F11fpk0hcbt?authuser=0&hl=en&rclk=1
9,The Smart Rehab,NaN,,NaN,https://www.google.com/maps/place/The+Smart+Rehab/data=!4m6!3m5!1s0x87d8cc3fad461b03:0xe97465982c39aee4!8m2!3d38.621353!4d-90.4280759!16s%2Fg%2F11gfgb__p8?authuser=0&hl=en&rclk=1


In [ ]:
20%5

In [ ]:
//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[3]/div/div[2]/div[2]/div[1]/div/div/div/div[4]/div[1]/span[1]/jsl/span[2]

In [ ]:
map_query = 'https://www.google.com/maps/search/Physical+Thrapy+Clinic+Saint+Louis+County+Missouri'
csv_file = "tailor_list.csv"
csv_columns = ["name","addr1","addr2","phone","web","loc","aux"]

def clean_tailor_info_panel(ugly_text_blob):
# assumes the first two lines are probably english and arabic addresses
# will check if 3rd line is opening time (it won't save it - just checks, if it looks like anything else it saves it in
#check if 4th / 5th line is a website or a phone number - save accordingly
# check if 5th/6th line is a "plus code" location or phone number and saves accordingly
    text_blob = ugly_text_blob.splitlines()
    addr = []
    website = ""
    phone = ""
    location = ""
    aux_unknown = ""
    for line in range(len(text_blob)):
        if line <= 1:
            addr.append(text_blob[line])
        elif line >= 2:
            if "closed" in text_blob[line].lower() or "open" in text_blob[line].lower() or "claim this" in text_blob[line].lower():
                aux_unknown
            else:
                if text_blob[line].startswith("+966") or text_blob[line].startswith("966"):
                    phone = text_blob[line]
                elif "+" in text_blob[line] and (re.search('[a-zA-z]', text_blob[line]) != None):
                    location = text_blob[line]
                else:
                    website = text_bolb[line]
    print ("Details:")
    for address in addr:
        print (f"Address: {address[0:5]}...")
    print(f"Phone:    {phone}")
    print (f"Loc:     {phone}")
    print (f"Website: {website}")
    print (f"Aux:     {aux_unknown}")
    print("------------------")

    cleaned_block = {"addr1" :addr[0], "addr2" :addr[1] if len(addr)>1 else "", "phone":phone,"loc":location,
                     "web" :website, "aux":aux_unknown}
    return cleaned_block


def google_map_extractor(driver, url):
    driver.get(url)
    more_pages = True
    master_tailor_array = []
    
    ctr = 0
    
    while more_pages:
        time.sleep(NEXT_PAGE_DELAY)
        # get the HTML data of the page
        html = driver.page_source
        # make sure that all the divs of the page have fully loaded
        divs = WebDriverWait(driver,5).until(EC.presence_of_all_elements_located ((By.CSS_SELECTOR,"div")))
        # Beautiful Soup (BS) is a convenient way to find stuff in a webpage
        soup = BeautifulSoup(html, "html.parser")
        time.sleep(WAIT_AFTER_ELEMENT_LOADED)
        # using (BS) look for a block called section result - I'll show in the video how I found this name.
        # Each result has a (different) value inside it called data-section-id, which I will use later to find
        # individual results and click on them. I picked that one because it is random and unique to each business
        # which may not be true if I use the name - like, it looks like there are one billion Mashar Tailors on every street...
        result_list = soup.find_all( 'div', attrs={'class': 'section-result' })
        # this one was a weird one - I'll explain in the video, but the idea is, you cannot click on something
        # if that is not "visible" on screen - so if you have a bunch of results, you have to be able to scroll them
        # so the browser thinks they are visible, before you can click them
       ###### result_scroll_panel = driver.find_element_by_css_selector("div[class^='section-layout section-scrollbox']")
        
        result_scroll_panel = driver.find_element_by_xpath ("//div[@  aria-label='Results for Physical Thrapy Clinic Saint Louis County Missouri']") 
        # just print out the number of results so you know nothing stupid has happened
        print (f"Num results on this page : {len(result_list)}")
        # for each of the (usually 20) results, we'll go and get their details
        for result_val in result_list:
        #data-result-Index' is the index of the result (i.e. 1 to 20)
        #"aria-label' is the actual name of the business, so MASHOOR, or Ladies Tailor etc
            print(f"\t {result_val['data-result-index']} ({result_val['aria-label']})")
            name = result_val['aria-label']
        
        
        
            #Assuming you've played around with basic HTML, we will look for
            #which has a the data-section-id I mentioned on line 96 above
            # we'll do 2 things with it:
            # 1) use it to fake "scroll" to make sure it is visible - you may not see it, but the browser will think it is visibl
            # 2) click on it to get the detail page for that specific tailor
            # find the exact div we want
            this_result_div = driver.find_element_by_xpath("//div[@data-section-id-'"+ result_val["data-section-id"] + "']")
            # first, we "scroll" to to make sure it is visible
            driver.execute_script("arguments[0].scrollIntoview();", this_result_div)
            # If not this result div.is displayed() :
            #print (fCant see (result val'aria-label']} - gonna scroll down a bit")
            #driver.execute_script("arguments[@].scrollIntoview();", this_result div)
            #time.sleep(1)
            # second, click on the result
            ActionChains(driver).click(this_result_div).perform()
            # now wait for the detail page to be loaded
            time.sleep(LOAD_SPECIFIC_RESULT_DELAY)
            # first look for the "back to list" button - which is a blue text area on top of the screen
            # that says "Back to Results"
            back_btn = driver.find_element_by_xpath("//button[@class='section-back-to-list-button blue-link noprint']")
            # get this new page
            this_tailor_html = driver.page_source
            # look for the panel that has all the detail crap about this tailor (its called 'section layout")
            this_tailor_info_panel = driver.find_element_by_xpath("//div[@class='section-layout']")
            # you can get all the details as a giant blob of text
            # but you have to clean it up into phone numbers, addresses, websites etc
            # so I made a small function called clean_tailor info_panel just above this to do that.
            cleaned_data = clean_tailor_info_panel(this_tailor_info_panel.text)
            # this has everything except the name, so I'11 just put the name (which we got at from the main list)
            # back into the data set.
            cleaned_data["name"] = name
            #print ("Cleaned Data:"", cleaned_ data)
            #print("csv_cols:", cv_ columns)
            # This is a little dumb, but I'm writing each result to the cv file as soon as a get it
            # its smarter to wait and get (for example) 20 results or whatever, a bunch of results and write it all at once
            # but if anything goes wrong, and it crashes, you'll lose anything that wasn't written to file.
            # we can do smarter stuff but honestly, its pretty boring and you will never notice that this is maybe 0.001 sece
            try:
                with open(csv_file,"a", encoding='utf-8') as csvfile:
                    writer = csv.DictWriter(csvfile, fieldnames=csv_columns, delimiter="|")
                    #writer writerow: writerow
                    writer.writerow(cleaned_data)
            except IOError:
                print("I/0 error while trying to write to csv file")
            # Once we've done this tailor's details - click on that blue back link we saved on line 139
            # to go back to the main list and wait for a few sec to load it again
            ActionChains (driver).click(back_btn).perform()
            time.sleep(BACK_TO_MAIN_LIST_DELAY)

            # once you've done all results on this page, find the "next page" button and click it
            # and then repeat the whole damn thing again
            next_btn = driver.find_element_by_xpath('//*[contains(@id,"sectich-pagination-button-next")]')
            # if there's no Next button, you've reached the last results page
            # no need to continule
            if next_btn is None:
                more_pages = False
                break
            time.sleep(NEXT_PAGE_DELAY)
            ActionChains(driver).click(next_btn).perform()
            print ("Next Clicked")

driver = webdriver.Chrome()
time.sleep(STARTUP_DELAY)
google_map_extractor(driver,map_query )

In [ ]:
df_level_1 = pd.DataFrame(columns=["LocName","LocRate","LocBussType","LocBussAdd","LocHREF"])
junk_url_list = []


# keep_looping=True
# while  keep_looping:

# for prefix in raw_text:
#     t1 = pd.Timestamp.now()
#     df_level_1.to_pickle('Physical_Therapy_MO_level_1_scrape_intermediate.pkl')
#     prefix = prefix.replace(' ',"+")
#     print(prefix)
#     for search_string in search_string_list:
#         search_string = search_string.replace(' ',"+")
#         print(search_string)

#         url_local = search_string + '+' + prefix
#         url_local_b = 'https://www.google.com/maps/search/' + url_local
#         print(url_local_b)
#         driver.get(url_local_b)



driver.get(r'https://www.google.com/maps/search/Physical+Thrapy+Clinic+Saint+Louis+County+Missouri')

driver.maximize_window()

        

action = webdriver.ActionChains(driver)
python_button = driver.find_element_by_xpath('//*[@aria-label=" Next page "]')
action.move_to_element(python_button)
action.perform()
action.move_by_offset(35, -50)
action.double_click()
action.perform()
time.sleep(.25)
action.double_click()
action.perform()
time.sleep(.25)
action.double_click()
action.perform()
time.sleep(.25)
action.double_click()
action.perform()
time.sleep(.25)

driver.find_element_by_xpath('//*[@aria-label="Results for Physical Thrapy Clinic Saint Louis County Missouri"]')
html = driver.page_source
soup = BeautifulSoup(html, "lxml")
# print(soup.prettify())
# tailes = driver.find_elements_by_tag_name("div")
# for tail in tailes:
#     print(tail.text)
        
# next_Page_button = driver.find_element_by_xpath('//*[@aria-label=" Next page "]')
# next_Page_button.click

In [ ]:
df_level_1 = pd.DataFrame(columns=["LocName","LocRate","LocBussType","LocBussAdd","LocHREF"])
junk_url_list = []


# keep_looping=True
# while  keep_looping:

for prefix in raw_text:
    t1 = pd.Timestamp.now()
    df_level_1.to_pickle('Physical_Therapy_MO_level_1_scrape_intermediate.pkl')
    prefix = prefix.replace(' ',"+")
    print(prefix)
    for search_string in search_string_list:
        search_string = search_string.replace(' ',"+")
        print(search_string)

        url_local = search_string + '+' + prefix
        url_local = 'https://www.google.com/maps/search/' + url_local
        print(url_local)
        driver.get(url_local)

        time.sleep(1)
        
        
#         keep_looping = True
#         pdx = 0
#         while keep_looping:

    #         print(soup)

#             for button_tmp in soup.find_all('button'):
#                 try:

#                     if button_tmp['aria-label'] == ' Next page ':
#                         button_id = button_tmp['id']

#                 except:
#                     pass

        
            python_button = driver.find_element_by_id(button_id)
            action.move_to_element(python_button)
            action.perform()
            action.move_by_offset(35, -50)
            action.double_click()
            action.perform()
            time.sleep(.25)
            action.double_click()
            action.perform()
            time.sleep(.25)
            action.double_click()
            action.perform()
            time.sleep(.25)
            action.double_click()
            action.perform()
            time.sleep(.25)


#             page_content = driver.page_source
#             print(page_content)

#   
#             response = Selector(page_content)
#             print(response)
            
#             with open("response.html","w", encoding="utf-8") as file:
#                 file.write(response.text)
#             try:
#                 python_button = driver.find_element_by_id(button_id)
#                 python_button.click()
#                 time.sleep(0.75)
#             except:
#                 #print('reached end of good buttons')
#                 keep_looping = False

#                 pdx+=1
#             if pdx > 10:
#                 keep_looping = False                

            
            
            
            
#             results = []
#             idx = 0

#             for el in response.xpath('//div[contains(@aria-label, "Results for")]/div/div[./a]'):
#                 LocHREF = el.xpath('./a/@href').extract_first('')
#                 df_level_1.loc[idx,"LocHREF"] = LocHREF

#                 LocName= el.xpath('./a/@aria-label').extract_first('')
#                 df_level_1.loc[idx,"LocName"] = LocName

#                 idx += 1

                
            
                
                
                
#     #             nextpage = driver.find_element_by_xpath("/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div/button[2]/img")
#     #             nextpage.click()
#     #             time.sleep(5)


#     #             results.append({
#     #                 'link': el.xpath('./a/@href').extract_first(''),
#     #                 'title': el.xpath('./a/@aria-label').extract_first(''),
#     #                 'rate': el.xpath('//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[6]/div/div[2]/div[2]/div[1]/div/div/div/div[3]/div/span[2]/span[2]').extract_first(''),
#     #                 'BusType': el.xpath('/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[26]/div/div[2]/div[2]/div[1]/div/div/div/div[4]/div[1]/span[1]/jsl/span[2]').extract_first('')

#     #         })

#     #         print(results)
# df_level_1

In [ ]:
driver.get("https://www.google.com/maps/place/Bengaluru,+Karnataka/@12.95396,77.4908522,11z/data=!4m5!3m4!1s0x3bae1670c9b44e6d:0xf8dfc3e8517e4fe0!8m2!3d12.9715987!4d77.5945627")
option = driver.find_element_by_xpath("//div[@class='hNLpDc-HiaYvf-DWDkFd-HiaYvf-haAclf']")
driver.execute_script("arguments[0].scrollIntoView(true);",option)

In [ ]:
# This part works!

df_level_1 = pd.DataFrame(columns=["LocName","LocRate","LocBussType","LocBussAdd","LocHREF"])
junk_url_list = []


idx = 0

for prefix in raw_text:
    t1 = pd.Timestamp.now()
    df_level_1.to_pickle('Physical_Therapy_MO_level_1_scrape_intermediate.pkl')
    prefix = prefix.replace(' ',"+")
    print(prefix)
    for search_string in search_string_list:
        search_string = search_string.replace(' ',"+")
        print(search_string)

        url_local = search_string + '+' + prefix
        url_local = 'https://www.google.com/maps/search/' + url_local
        print(url_local)
        driver.get(url_local)

        time.sleep(1)

        action = webdriver.ActionChains(driver)
        soup = BeautifulSoup(driver.page_source, 'lxml')
#         print(soup)

        for button_tmp in soup.find_all('button'):
            try:

                if button_tmp['aria-label'] == ' Next page ':
                    button_id = button_tmp['id']

            except:
                pass

        python_button = driver.find_element_by_id(button_id)
        action.move_to_element(python_button)
        action.perform()
        action.move_by_offset(35, -50)
        action.double_click()
        action.perform()
        time.sleep(.25)
        action.double_click()
        action.perform()
        time.sleep(.25)
        action.double_click()
        action.perform()
        time.sleep(.25)
        action.double_click()
        action.perform()
        time.sleep(.25)

        soup = BeautifulSoup(driver.page_source, 'lxml')

#         print(soup)

        time.sleep(1)

    
    
        review_titles=driver.find_element_by_xpath(r'/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[4]/div/a')
        print(review_titles.text)
#         idx = 0

#         all_tiles = soup.find_all('div')
#         for tile in all_tiles:
# #             print(tile)
#             LocName  =  tile.find('a').get('aria-label')
#             df_level_1.loc[idx,"LocName"] = LocName

#             LocRate     = tile.find('span').get('aria-label')
#             df_level_1.loc[idx,"LocRate"] = LocRate

#             LocBussType = tail.find_all('span', attrs={"jstcache":"131"})[0].text
#             df_level_1.loc[idx,"LocBussType"] = LocBussType

#             LocBussAdd  = tail.find_all('span', attrs={"jstcache":"131"})[1].text
#             df_level_1.loc[idx,"LocBussAdd"] = LocBussAdd

#             LocHREF     = tile.find("a").get('href')
#             df_level_1.loc[idx,"LocHREF"] = LocHREF
            
#             idx +=1

#             print(t1, " - ",LocName," - ", LocRate, " - ",LocBussType," - ",LocBussAdd," - ",LocHREF)

        
        
# df_level_1        

In [ ]:
tile.text

In [ ]:
# This part works!

df_level_1 = pd.DataFrame(columns=["LocName","LocRate","LocBussType","LocBussAdd","LocHREF"])
junk_url_list = []


idx = 0

for prefix in raw_text:
    t1 = pd.Timestamp.now()
    df_level_1.to_pickle('Physical_Therapy_MO_level_1_scrape_intermediate.pkl')
    prefix = prefix.replace(' ',"+")
    print(prefix)
    for search_string in search_string_list:
        search_string = search_string.replace(' ',"+")
        print(search_string)

        url_local = search_string + '+' + prefix
        url_local = 'https://www.google.com/maps/search/' + url_local
        print(url_local)
        driver.get(url_local)

        time.sleep(1)

        action = webdriver.ActionChains(driver)
        soup = BeautifulSoup(driver.page_source, 'lxml')
#         print(soup)

        for button_tmp in soup.find_all('button'):
            try:

                if button_tmp['aria-label'] == ' Next page ':
                    button_id = button_tmp['id']

            except:
                pass

        python_button = driver.find_element_by_id(button_id)
        action.move_to_element(python_button)
        action.perform()
        action.move_by_offset(35, -50)
        action.double_click()
        action.perform()
        time.sleep(.25)
        action.double_click()
        action.perform()
        time.sleep(.25)
        action.double_click()
        action.perform()
        time.sleep(.25)
        action.double_click()
        action.perform()
        time.sleep(.25)

        soup = BeautifulSoup(driver.page_source, 'lxml')

#         print(soup)

        time.sleep(1)


        idx = 0

        all_tails = soup.find_all('div', attrs={"jstcache": "214"})
        for tail in all_tails:
            LocName     = tail.find_all('span', attrs={"jstcache":"100"})[0].text
            df_level_1.loc[idx,"LocName"] = LocName

            LocRate     = tail.find_all('span',attrs={"jstcache":"103"})[0].text
            df_level_1.loc[idx,"LocRate"] = LocRate

            LocBussType = tail.find_all('span', attrs={"jstcache":"131"})[0].text
            df_level_1.loc[idx,"LocBussType"] = LocBussType

            LocBussAdd  = tail.find_all('span', attrs={"jstcache":"131"})[1].text
            df_level_1.loc[idx,"LocBussAdd"] = LocBussAdd

            LocHREF     = tail.find("a").get('href')
            df_level_1.loc[idx,"LocHREF"] = LocHREF
            
            idx +=1

            print(t1, " - ",LocName," - ", LocRate, " - ",LocBussType," - ",LocBussAdd," - ",LocHREF)

        
        
df_level_1        

In [ ]:
# This cell does not work correctly. It is the same code as the above, but desigened 
#to get data from first 10 pages of google map search.

df_level_1 = pd.DataFrame(columns=["LocName","LocRate","LocBussType","LocBussAdd","LocHREF"])
junk_url_list = []


keep_looping=True
while  keep_looping:
    for prefix in raw_text:
        t1 = pd.Timestamp.now()
        df_level_1.to_pickle('Physical_Therapy_MO_level_1_scrape_intermediate.pkl')
        prefix = prefix.replace(' ',"+")
        print(prefix)
        for search_string in search_string_list:
            search_string = search_string.replace(' ',"+")
            print(search_string)

            url_local = search_string + '+' + prefix
            url_local = 'https://www.google.com/maps/search/' + url_local
            print(url_local)
            driver.get(url_local)

            time.sleep(1)
            
#             pdx = 0  # for google map pages
            idx = 0  # for rows

            soup = BeautifulSoup(driver.page_source, 'lxml')
            for button_tmp in soup.find_all('button'):
                try:
                    if button_tmp['aria-label'] == ' Next page ':
                        button_id = button_tmp['id']
                except:
                    pass
            action = webdriver.ActionChains(driver)
            python_button = driver.find_element_by_id(button_id)
            action.move_to_element(python_button)
            action.perform()
            action.move_by_offset(35, -50)
            action.double_click()
            action.perform()
            time.sleep(.25)
            action.double_click()
            action.perform()
            time.sleep(.25)
            action.double_click()
            action.perform()
            time.sleep(.25)
            action.double_click()
            action.perform()
            time.sleep(.25)

            soup = BeautifulSoup(driver.page_source, 'lxml')

#         print(soup)

            time.sleep(1)

            all_tails = soup.find_all('div', attrs={"jstcache": "206"})
            for tail in all_tails:
                LocName     = tail.find_all('span', attrs={"jstcache":"100"})[0].text
                df_level_1.loc[idx,"LocName"] = LocName

                LocRate     = tail.find_all('span',attrs={"jstcache":"103"})[0].text
                df_level_1.loc[idx,"LocRate"] = LocRate

                LocBussType = tail.find_all('span', attrs={"jstcache":"131"})[0].text
                df_level_1.loc[idx,"LocBussType"] = LocBussType

                LocBussAdd  = tail.find_all('span', attrs={"jstcache":"131"})[1].text
                df_level_1.loc[idx,"LocBussAdd"] = LocBussAdd

                LocHREF     = tail.find("a").get('href')
                df_level_1.loc[idx,"LocHREF"] = LocHREF

                print(t1, " - ",LocName," - ", LocRate, " - ",LocBussType," - ",LocBussAdd," - ",LocHREF)

                idx +=1

#             nextpage = driver.find_element_by_xpath("/html/body/div[3]/div[9]/div[9]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[2]/div/div[1]/div/button[2]/img")
#             nextpage.click()
#             time.sleep(5)
#     idx +=1       
# pdx +=1
# try:
#     if pdx >10:
#         keep_looping = False
# except:
#     pass


        
        
df_level_1        

In [ ]:
twint

In [ ]:
import winsound as ws
for i in range(1):
    ws.Beep(2000,1000)